# Price Check Notebook

Simple GenAI application that queries an URL that contains a product and extract the price, size and name of the product.

## Imports

In [2]:
!pip install openai langchain

In [4]:
import openai
openai.api_key = ''

In [6]:
from langchain.chat_models import ChatOpenAI

In [7]:
from langchain.schema import AIMessage, HumanMessage, SystemMessage

In [52]:
llm = ChatOpenAI(openai_api_key=openai.api_key, model_name="gpt-3.5-turbo")

### Prompt Engineering

Giving the instructions on how the model should behave including the JSON Schema and a few show example.

In [53]:
system_message = SystemMessage(content="""
You're an assistant that given a list of urls containing a product in a marketplace, 
you will be able to retrieve the name, price of the product and the sizes available. 
Please return the data as list in the following json schema: 

[
    {
        {
            "title": "product_name",
            "content": "string containing the product name"
        },
        {
            "title": "product_sizes",
            "content": "array containing the available product sizes"
        },
        {
            "title": "product_price",
            "content": "string containing the product price in BRL format"
        }
    }
]

Here is an example of the output:

[
    {
        "product_name": "Camisa Corinthians",
        "product_sizes": ["P", "3G"],
        "product_price": "R$ 123,00"
    },
    {
        "product_name": "Camisa Dois",
        "product_sizes": ["GG"],
        "product_price": "R$ 333,00"
    }
]


If you're not able to return the values, please return false.
""")

In [27]:
user_message = HumanMessage(content="Can you please check and return the expected information from this website: https://www.nike.com.br/camisa-nike-corinthians-iii-2023-24-torcedor-pro-masculina-027117.html?cor=0L ?")

In [31]:
reply = llm([
 system_message,
 user_message
])

In [32]:
reply

AIMessage(content='{\n "product_name": "Camisa Nike Corinthians III 2023/24 Torcedor Pro Masculina",\n "product_sizes": ["P", "M", "G", "GG"],\n "product_price": "R$ 279,90"\n}')

### Converting the Output

In [34]:
import json

In [35]:
json_response = json.loads(reply.content)

In [36]:
json_response

{'product_name': 'Camisa Nike Corinthians III 2023/24 Torcedor Pro Masculina',
 'product_sizes': ['P', 'M', 'G', 'GG'],
 'product_price': 'R$ 279,90'}

In [63]:
urls = ["https://www.adidas.com.br/camiseta-manchester-united/IP1746.html", "https:/br.puma.com/camisa-manchester-city-23-24-third-masculina-770460-03.html", "https://www.nike.com.br/camisa-nike-corinthians-iii-2023-24-torcedor-pro-masculina-027117.html?cor=0L"]

string_urls = "\n".join(urls)

user_message = HumanMessage(content=f"""
Can you please check and return the expected information from these websites:
{string_urls}
""")

reply = llm([
 system_message,
 user_message
])

reply



AIMessage(content='{\n    "data": [\n        {\n            "product_name": "Camiseta Manchester United",\n            "product_sizes": ["P", "M", "G", "GG"],\n            "product_price": "R$ 249,99"\n        },\n        {\n            "product_name": "Camisa Manchester City 23/24 Third Masculina",\n            "product_sizes": ["P", "M", "G", "GG"],\n            "product_price": "R$ 299,90"\n        },\n        {\n            "product_name": "Camisa Nike Corinthians III 2023/24 Torcedor Pro Masculina",\n            "product_sizes": ["P", "M", "G", "GG"],\n            "product_price": "R$ 349,99"\n        }\n    ]\n}')

In [65]:
json_response = json.loads(reply.content)
json_response

{'data': [{'product_name': 'Camiseta Manchester United',
   'product_sizes': ['P', 'M', 'G', 'GG'],
   'product_price': 'R$ 249,99'},
  {'product_name': 'Camisa Manchester City 23/24 Third Masculina',
   'product_sizes': ['P', 'M', 'G', 'GG'],
   'product_price': 'R$ 299,90'},
  {'product_name': 'Camisa Nike Corinthians III 2023/24 Torcedor Pro Masculina',
   'product_sizes': ['P', 'M', 'G', 'GG'],
   'product_price': 'R$ 349,99'}]}